In [1]:
from dolfin import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Set up the mesh and create finite element space relative to the mesh
mesh = UnitSquareMesh(32, 32, "crossed")
V = FunctionSpace(mesh, "Lagrange", 2)

In [3]:
# Define boundary condition u = 0 at x = 0, u = 1 at x = 1.
def left_boundary(x, on_boundary):
    return near(x[0], 0, DOLFIN_EPS_LARGE) and on_boundary

def right_boundary(x, on_boundary):
    return near(x[0], 1.0, DOLFIN_EPS_LARGE) and on_boundary

Gamma_0 = DirichletBC(V, Constant(0.0), left_boundary)
Gamma_1 = DirichletBC(V, Constant(1.0), right_boundary)
bcs = [Gamma_0, Gamma_1]

In [4]:
# Define q(u) = (1+u)^m and its derivative Dq(u) = m*(1+u)^(m-1)
m = 2
def qq(u):
    return (1+u)**m

def Dqq(u):
    return m*(1+u)**(m-1)

u_ = Function(V)
v = TestFunction(V)

# The trial function is now the incremental solution du
du = TrialFunction(V)
v = TestFunction(V)
u_ = Function(V)    # the most recently computed solution
# Define the nonlinear problem in the variational formulation
F = dot(qq(u_) * grad(u_), grad(v))*dx
# Expression for the Gateaux derivative of F
Jacobian = dot(qq(u_) * grad(du), grad(v))*dx \
            + dot(Dqq(u_)*du * grad(u_), grad(v))*dx

In [6]:
problem = NonlinearVariationalProblem(F, u_, bcs, Jacobian)
solver = NonlinearVariationalSolver(problem)
prm = solver.parameters

In [7]:
prm["newton_solver"]["absolute_tolerance"] = 1E-12
prm["newton_solver"]["relative_tolerance"] = 1E-10
prm["newton_solver"]["maximum_iterations"] = 50
prm["newton_solver"]["relaxation_parameter"] = 1.0

iterative_solver = False
if iterative_solver:
    prm["linear_solver"] = "gmres"
    prm["preconditioner"] = "ilu"
    prm["krylov_solver"]["absolute_tolerance"] = 1E-9
    prm["krylov_solver"]["relative_tolerance"] = 1E-7
    prm["krylov_solver"]["maximum_iterations"] = 1000
    prm["krylov_solver"]["gmres"]["restart"] = 40
    prm["krylov_solver"]["preconditioner"]["ilu"]["fill_level"] = 0
    
solver.solve()

(0, True)

In [ ]:
u_exact = Expression("pow(7*x[0] + 1, 1.0/3) - 1", degree = 3)

plt.figure(figsize=(10, 3), dpi=80)
plt.subplot(1, 2, 1)
plt.colorbar(plot(u_))
plt.subplot(1, 2, 2)
plt.colorbar(plot(u_ - project(u_exact, V)))

In [ ]:
# Alterantively, we can compute the Jacobian by symbolic computation with
# derivative(F, u_, du). The above problem setup can be written compactly as
# follows:
du = TrialFunction(V)
DF = derivative(F, u_, du)
solve(F == 0, u_, bcs, J=DF)

u_exact = Expression("pow(7*x[0] + 1, 1.0/3) - 1", degree = 3)

plt.figure(figsize=(10, 3), dpi=80)
plt.subplot(1, 2, 1)
plt.colorbar(plot(u_))
plt.subplot(1, 2, 2)
plt.colorbar(plot(u_ - project(u_exact, V)))